In [2]:
# prepare dataloders for training
import os
import torch
import importlib
import sys
sys.path.append('..')
import data_pre
import warnings
warnings.filterwarnings("ignore")

In [4]:
# 准备数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "data_cmapss"  # 
file_names = ["train_FD004.txt"]  #
for i in range(1, 6):
    latent_dim = i
    class_dim = latent_dim
    dim = latent_dim
    importlib.reload(data_pre)
    cmapss = data_pre.CmapssData(file_path, file_names, class_dim=class_dim, latent_dim=latent_dim)
    t_model_con_data = torch.tensor(cmapss.context_data.values).float().to(DEVICE)
    t_model_sensor_data = torch.tensor(cmapss.sensor_data.values).float().to(DEVICE)
    r_input_init = torch.tensor(cmapss.r_input_init*0).float().to(DEVICE)
    # 需要空值 mask 的 ratio
    label_input = r_input_init
    train_ratio = 0.5
    batch_size = 256*3-1
    loader, set = data_pre.get_train_test_valid(train_ratio, batch_size, t_model_sensor_data, t_model_con_data, label_input, torch.tensor(cmapss.train_label), torch.tensor(cmapss.index_data[['unit_id', 'cycle']].values))
    data_dir = os.path.join("dataloaders")
    data_path = os.path.join(data_dir, str(i))
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    torch.save([loader, set], os.path.join(data_path, "cmapss_004.pt"))

In [5]:
sensor_dim = 14#t_model_sensor_data.shape[-1]
context_dim = 3#t_model_con_data.shape[-1]
depth = 3
encoder_width = [50,50,200]
decoder_width = [200,50,50]
drop_rate = 0.0

def prepare_params(depth, encoder_width, decoder_width, drop_rate, sensor_dim, context_dim):
    # most basic parameters for model
    encoder_params = {'encoder_depth':depth,
                    'encoder_width': encoder_width,
                    'drop_rate': drop_rate}
    decoder_params = {'decoder_depth':depth,
                    'decoder_width': decoder_width,
                    'drop_rate': drop_rate,}
    encoder_params['sensor_dim'] = sensor_dim
    encoder_params['context_dim'] = context_dim
    decoder_params['sensor_dim'] = sensor_dim
    decoder_params['context_dim'] = context_dim

    return encoder_params, decoder_params


In [6]:
# prepare lrd parameters
# lrd: low rank discriminative model
kl_weights = 0.01
encoder_params, decoder_params = prepare_params(depth, encoder_width, decoder_width,drop_rate, 
                                                               sensor_dim, context_dim)


encoder_params['mu_activation'] = 'sigmoid'
decoder_params['drop_rate'] = 0.1
decoder_params['decoder_depth'] = 2
decoder_params['decoder_width'] = [200,50]
decoder_params['recons_activation'] = 'None'

encoder_params['encoder_activation'] = 'leaky_relu'
decoder_params['decoder_activation'] = 'leaky_relu'
encoder_params['cov_activation'] = 'sigmoid'
encoder_params['muzinline'] = 1
cmapss_lrd_params = {'encoder_params': encoder_params,
             'decoder_params': decoder_params,
             'kl_weights': kl_weights,}


import os
base_dir = 'model_params'
        
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
            
params_path = os.path.join(base_dir, 'cmapss_lrd.yaml')

import yaml

with open(params_path, 'w') as file:
    yaml.dump(cmapss_lrd_params, file)

In [7]:
import code_package.model_package as mcr2_model
params_path = os.path.join('model_params', 'cmapss_lrd.yaml')
mnist_lrd_params = yaml.load(open(params_path, 'r'), Loader=yaml.FullLoader)

class_dim = 10
latent_dim = 10
encoder_params = mnist_lrd_params['encoder_params']
decoder_params = mnist_lrd_params['decoder_params']
kl_weights = mnist_lrd_params['kl_weights']
encoder_params['class_dim'] = class_dim
encoder_params['latent_dim'] = latent_dim
decoder_params['class_dim'] = class_dim
decoder_params['latent_dim'] = latent_dim
lambdas = 1e-20
mcr2 =  mcr2_model.MCR2(encoder_params, decoder_params, is_ts=0, kl_weights=kl_weights)
mcr2.lambdas = lambdas
mcr2.to('cuda:0')

MCR2(
  (flatten_layer): Flatten(start_dim=1, end_dim=-1)
  (encoder): MCR2Encoder(
    (encoder): Sequential(
      (layer0): Linear(in_features=17, out_features=50, bias=True)
      (dropout0): Dropout(p=0.0, inplace=False)
      (relu0): LeakyReLU(negative_slope=0.01)
      (layer1): Linear(in_features=50, out_features=50, bias=True)
      (dropout1): Dropout(p=0.0, inplace=False)
      (relu1): LeakyReLU(negative_slope=0.01)
      (layer2): Linear(in_features=50, out_features=200, bias=True)
      (dropout2): Dropout(p=0.0, inplace=False)
      (relu2): LeakyReLU(negative_slope=0.01)
    )
    (direct): Sequential(
      (direct): Linear(in_features=200, out_features=10, bias=True)
    )
    (cov_vec_layer): Sequential(
      (cl): Linear(in_features=200, out_features=1, bias=True)
      (c_ac): Sigmoid()
    )
    (mu_layer): Sequential(
      (ml): Linear(in_features=200, out_features=1, bias=True)
      (m_ac): Sigmoid()
    )
  )
  (decoder): Decoder(
    (decoder): Sequential(

In [8]:
# prepare cvae/vade parameters

encoder_params, decoder_params = prepare_params(depth, encoder_width, decoder_width,drop_rate, 
                                                               sensor_dim, context_dim)

encoder_params['mu_activation'] = 'None'
decoder_params['drop_rate'] = 0.1
decoder_params['decoder_depth'] = 2
decoder_params['decoder_width'] = [200,50]
decoder_params['recons_activation'] = 'None'

encoder_params['encoder_activation'] = 'leaky_relu'
decoder_params['decoder_activation'] = 'leaky_relu'
encoder_params['cov_activation'] = 'None'

cmapss_ae_params = {'encoder_params': encoder_params,
             'decoder_params': decoder_params,
             'kl_weights': kl_weights,}

import os
base_dir = 'model_params'
        
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
            
params_path = os.path.join(base_dir, 'cmapss_ae.yaml')

import yaml

with open(params_path, 'w') as file:
    yaml.dump(cmapss_ae_params, file)

In [9]:
# 最终使用版本
params_path = os.path.join('model_params', 'cmapss_ae.yaml')
mnist_ae_params = yaml.load(open(params_path, 'r'), Loader=yaml.FullLoader)

class_dim = 10
latent_dim = 10
encoder_params = mnist_ae_params['encoder_params']
decoder_params = mnist_ae_params['decoder_params']
kl_weights = 0.001  #mnist_ae_params['kl_weights']
encoder_params['class_dim'] = class_dim
encoder_params['latent_dim'] = latent_dim
decoder_params['class_dim'] = class_dim
decoder_params['latent_dim'] = latent_dim

mcr2 =  mcr2_model.CVAE(encoder_params, decoder_params, is_ts=0, kl_weights=kl_weights)
mcr2.to('cuda:0')

CVAE(
  (flatten_layer): Flatten(start_dim=1, end_dim=-1)
  (encoder): CVAEEncoder(
    (encoder): Sequential(
      (layer0): Linear(in_features=17, out_features=50, bias=True)
      (dropout0): Dropout(p=0.0, inplace=False)
      (relu0): LeakyReLU(negative_slope=0.01)
      (layer1): Linear(in_features=50, out_features=50, bias=True)
      (dropout1): Dropout(p=0.0, inplace=False)
      (relu1): LeakyReLU(negative_slope=0.01)
      (layer2): Linear(in_features=50, out_features=200, bias=True)
      (dropout2): Dropout(p=0.0, inplace=False)
      (relu2): LeakyReLU(negative_slope=0.01)
    )
    (cov_vec_layer): Sequential(
      (cl): Linear(in_features=200, out_features=10, bias=True)
    )
    (mu_layer): Sequential(
      (ml): Linear(in_features=200, out_features=10, bias=True)
    )
  )
  (decoder): Decoder(
    (decoder): Sequential(
      (layer0): Linear(in_features=13, out_features=200, bias=True)
      (dropout0): Dropout(p=0.1, inplace=False)
      (acti0): LeakyReLU(nega

In [10]:
kl_weights = 0.001
config = {'gamma': 0.95,
        'learning_rate': 0.002,
        'kl_weights': kl_weights, 
        'n_epoch': 100,
        'save_freq': 150,
        'save_model_epoch': 0,
        'step_size': 10,
        'pre_train_epoch': 500,
        'weight_ls':[1,20,1,0],
        'mcr2':1, # 是否使用mcr2模型
        'pre_train':1, # 是否使用预训练
        'n_start':10
        }
        
if not os.path.exists('configs'):
        os.makedirs('configs')
# Save the config dictionary to a YAML file
config_path = os.path.join('configs', 'cmapss_mcr2_config.yaml')
import yaml

with open(config_path, 'w') as file:
        yaml.dump(config, file)

In [11]:
config = {'gamma': 0.95,
        'learning_rate': 0.001,
        'kl_weights': kl_weights, 
        'n_epoch': 600,
        'save_freq': 150,
        'save_model_epoch': 0,
        'step_size': 10,
        'pre_train_epoch': 300,
        'weight_ls':[1,1,1,0],
        'mcr2':0, # 是否使用mcr2模型
        'pre_train':0, # 是否使用预训练
        'n_start':5
        }   
        
if not os.path.exists('configs'):
        os.makedirs('configs')
# Save the config dictionary to a YAML file
config_path = os.path.join('configs', 'cmapss_cvae_config.yaml')
import yaml

with open(config_path, 'w') as file:
        yaml.dump(config, file)